## Project 10, task 4, Кривоногов Н.В. (hemerahu)

## 1. Создай соединение с базой данных с помощью библиотеки sqlite3.

In [1]:
import sqlite3

conn = sqlite3.connect('../data/checking-logs.sqlite')

conn

## 2. Получи схему таблицы test.

In [2]:
import pandas as pd

df = pd.io.sql.read_sql('PRAGMA table_info(test);', conn)

df

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,labname,TEXT,0,None,0
3,3,first_commit_ts,TIMESTAMP,0,None,0
4,4,first_view_ts,TIMESTAMP,0,None,0


## 3. Получи только первые 10 строк таблицы test, чтобы проверить, как она выглядит.

In [3]:
df_head_10 = pd.io.sql.read_sql('SELECT * FROM test LIMIT 10;', conn)

df_head_10

,index,uid,labname,first_commit_ts,first_view_ts
0,3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,18,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,19,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,20,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,21,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,23,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


## 4. Найди среди всех пользователей минимальное значение этой самой дельты (периода времени от даты первого коммита пользователя до срока сдачи соответствующего лабораторного задания), используя только один запрос.
- Для этого сджойни данные своей таблицы с данными таблицы deadlines.
- Период времени должен отображаться в часах.
- Не учитывай лабораторное задание project1 с более длительным сроком сдачи, поскольку оно будет выделяться на фоне других.
- Сохрани значение в датафрейме df_min с соответствующим uid.

In [4]:
df_min = pd.io.sql.read_sql('SELECT t.uid, CAST(MIN(JULIANDAY(t.first_commit_ts) - JULIANDAY(d.deadlines, "unixepoch")) * 24 \
                             AS INTEGER) AS "MIN(diff)" FROM test AS t LEFT JOIN deadlines as d ON t.labname = d.labs \
                             WHERE t.labname IN ("laba04", "laba04s", "laba05", "laba06", "laba06s");', conn)

df_min

,uid,MIN(diff)
0,user_30,-202


## 5. Выполни те же самые операции для максимального значения дельты, используя только один запрос. Название итогового датафрейма — df_max.

In [5]:
df_max = pd.io.sql.read_sql('SELECT t.uid, CAST(MAX(JULIANDAY(t.first_commit_ts) - JULIANDAY(d.deadlines, "unixepoch")) * 24 \
                             AS INTEGER) AS "MAX(diff)" FROM test AS t LEFT JOIN deadlines as d ON t.labname = d.labs \
                             WHERE t.labname IN ("laba04", "laba04s", "laba05", "laba06", "laba06s");', conn)

df_max

,uid,MAX(diff)
0,user_25,-2


## 6. Выполни те же самые операции для среднего значения дельты, используя только один запрос. На этот раз ваш итоговый датафрейм не должен включать столбец uid; имя датафрейма — df_avg.

In [6]:
df_avg = pd.io.sql.read_sql('SELECT AVG(CAST((JULIANDAY(t.first_commit_ts) - JULIANDAY(d.deadlines, "unixepoch")) * 24 \
                             AS INTEGER)) AS "AVG(diff)" FROM test AS t LEFT JOIN deadlines as d ON t.labname = d.labs \
                             WHERE t.labname IN ("laba04", "laba04s", "laba05", "laba06", "laba06s");', conn)

df_avg

,AVG(diff)
0,-89.125


## 7. Мы хотим проверить предположение о том, что у пользователей, посетивших ленту новостей всего несколько раз, период времени между датой первого коммита и сроком сдачи лабораторного задания меньше. Для этого тебе необходимо рассчитать коэффициент корреляции между количеством просмотров страниц и разницей между датами.
Используя только один запрос, создай таблицу со столбцами: uid, avg_diff, pageviews.
- uid — это uid, существующие в таблице test.
- avg_diff — среднее значение дельты (периода времени между датой первого коммита и сроком сдачи лабораторного задания) для каждого пользователя.
- pageviews — количество посещений ленты новостей одним пользователем.

Не учитывай лабораторное задание project1.

Сохрани результаты в датафрейме views_diff.

Используй метод corr() библиотеки Pandas для вычисления коэффициента корреляции между количеством просмотров и значением дельты.

In [7]:
views_diff = pd.io.sql.read_sql('SELECT t.uid, t.avg_diff, p.pageviews FROM \
                                (SELECT uid, AVG((STRFTIME("%s", first_commit_ts) - deadlines) / 3600) AS avg_diff \
                                FROM test LEFT JOIN deadlines ON test.labname = deadlines.labs \
                                WHERE test.labname IN ("laba04", "laba04s", "laba05", "laba06", "laba06s") GROUP BY uid) AS t \
                                LEFT JOIN \
                                (SELECT uid, COUNT(datetime) AS pageviews FROM pageviews WHERE uid LIKE "user_%" \
                                GROUP BY uid) AS p ON t.uid = p.uid;', conn)

views_diff.corr()

# предупреждение: в будущих версиях такой метод не будет работать, нужно будет вручную выбирать числовые признаки  

C:\Users\nick1\AppData\Local\Temp\ipykernel_7416\1514327407.py:9: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  views_diff.corr()


,avg_diff,pageviews
avg_diff,1.000000,-0.279736
pageviews,-0.279736,1.000000


## 8. Закрой соединение.

In [8]:
conn.close()